In [2]:
## for plotting SSH / ocean color underway with ship track 

In [3]:
import xarray as xr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from os import listdir
import glob
import os
import pandas as pd
import imageio
import scipy
from netCDF4 import Dataset
import datetime as dt
import cartopy.crs as ccrs

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy
import earthaccess
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import xarray as xr
import os
import netCDF4 
import fnmatch


from matplotlib.animation import FuncAnimation, PillowWriter  
import matplotlib.colors as mcolors

## Accessing NASA Ocean color files from shared drive 

In [4]:
# sort the files by date 

path = '/home/vboatwright/mnt/scienceparty_share/SR2503_scienceparty_share/SAT/L3m_CHL_NRT_StLucia/' # change to path to scienceshare on your local computer 
path = '/home/vboatwright/OneDrive/Documents/SIO/projects/santalucia/data/L3m_CHL_NRT_StLucia/'
path = '/media/vboatwright/KBZ/SR2503_scienceparty_share/SAT/L3m_CHL_NRT_StLucia/'

files = os.listdir(path)

# readable files: 
readable_files = []

for nc_file in files:
    try:
        with netCDF4.Dataset(path+nc_file, "r") as ds:
            readable_files.append(nc_file)
    except OSError as e:
        if "HDF error" in str(e):
            print(f"Skipping {nc_file} due to HDF error.")
        else:
            print(f"Skipping {nc_file} due to unexpected error: {e}")
    except Exception as e:
        print(f"Skipping {nc_file} due to general error: {e}")

# dates are sorted for these at characters 11:19 
sorted_files = sorted(readable_files, key=lambda x: x[11:19])



Skipping AQUA_MODIS.20250125_20250201.L3m.8D.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250201.L3m.DAY.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250202.L3m.DAY.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250202_20250209.L3m.8D.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250210_20250217.L3m.8D.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250218_20250225.L3m.8D.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250223.L3m.DAY.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250302.L3m.DAY.CHL.chlor_a.4km.NRT.nc due to HDF error.
Skipping AQUA_MODIS.20250302.L3m.DAY.CHL.chlor_a.9km.NRT.nc due to HDF error.


In [5]:
sorted_files

['AQUA_MODIS.20250125_20250201.L3m.8D.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250201.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250201_20250228.L3m.MO.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250201_20250228.L3m.MO.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250202.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250202_20250209.L3m.8D.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250203.L3m.DAY.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250203.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250204.L3m.DAY.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250204.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250205.L3m.DAY.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250205.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250206.L3m.DAY.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250206.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250207.L3m.DAY.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250207.L3m.DAY.CHL.chlor_a.9km.NRT.nc',
 'AQUA_MODIS.20250208.L3m.DAY.CHL.chlor_a.4km.NRT.nc',
 'AQUA_MODIS.20250208.L3m.DAY.CHL

In [6]:
daily4km_files

NameError: name 'daily4km_files' is not defined

In [ ]:
## animate 

import matplotlib.animation as animation
import cartopy.crs as ccrs
import matplotlib.ticker as mticker


# select for only the desired timestep satellite grabs 

daily9km_files = [f for f in sorted_files if fnmatch.fnmatch(f, '*DAY*9km*.nc')]
daily4km_files = [f for f in sorted_files if fnmatch.fnmatch(f, '*DAY*4km*.nc')]
comp8d_files = [f for f in sorted_files if fnmatch.fnmatch(f, '*8D*9km*.nc')]


# now perform for each day satellite file that i have 

# set bounds
xmin, xmax = -124, -117
ymin, ymax = 32, 36
sdlonlat = [-117.24556, 32.928333]
chl_extent = [0, 3]

# loop through all valid files
chl_data = []
titles = []
lon, lat = None, None  

res = '4km' # 9 or 4 km 
for ff in daily4km_files:
    dataset = xr.open_dataset(path+ff)
    chl = dataset["chlor_a"]
    if lon is None or lat is None:
        lon, lat = np.meshgrid(dataset["lon"], dataset["lat"])
    chl_data.append(chl.values)
    given_date = datetime.strptime(dataset.attrs.get("product_name", ff)[11:19], "%Y%m%d")
    titles.append(f'Daily {res} on {given_date.strftime("%Y-%m-%d")}')  

# check if we loaded data
if not chl_data:
    raise ValueError("No valid chlorophyll data found.")


fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
crs_data = ccrs.PlateCarree()

# create empty plot for animation 
mesh = ax.pcolormesh(lon, lat, chl_data[0], cmap="jet", vmin=chl_extent[0], vmax=chl_extent[1], transform=crs_data)
# log scale plot: 
# change mins: 
#chl_extent = [1e-2, 3]
#mesh = ax.pcolormesh(lon,lat,chl_data[0],cmap='jet',transform=crs_data,norm=mcolors.LogNorm(vmin=chl_extent[0],vmax=chl_extent[1]) )
cbar = plt.colorbar(mesh, ax=ax, orientation='vertical', pad=0.05)
cbar.set_label(r'Chlorophyll Concentration [mg/$m^3$]')

ax.set_extent((xmin, xmax, ymin, ymax))
ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.2, linestyle="-")
gl.top_labels = False
gl.right_labels = False
gl.xlocator = mticker.MultipleLocator(1)
gl.ylocator = mticker.MultipleLocator(1)

# reference SD 
ax.scatter(sdlonlat[0], sdlonlat[1], color="red", marker="*", label="SD")
title = ax.set_title(titles[0])

# update animation via function
def update(frame):
    mesh.set_array(chl_data[frame].ravel())  # update pcolormesh data
    title.set_text(titles[frame])  # update title
    return mesh, title

ani = animation.FuncAnimation(fig, update, frames=len(chl_data), interval=500, blit=False)
plt.show()

# save as GIF
#save_loc = '/home/vboatwright/OneDrive/Documents/SIO/projects/santalucia/at_sea/'
#ani.save(save_loc+'santalucia_daily_aquamodis_'+res+'.gif', writer="pillow", fps=1)

# save as MP4 - not currently loaded correctly
# ani.save("chlorophyll_animation.mp4", writer="ffmpeg", fps=5)




In [ ]:
comp8d9km_files = [f for f in sorted_files if fnmatch.fnmatch(f, '*8D*9km*.nc')][0]



In [ ]:
comp8d9km_files = [f for f in sorted_files if fnmatch.fnmatch(f, '*8D*.nc')]

comp8d9km_files

In [ ]:
comp8d9km_files = [f for f in sorted_files if fnmatch.fnmatch(f, '*8D*9km*.nc')]
dataset = xr.open_dataset(path+ff) 
chl = dataset["chlor_a"]
clon, clat = np.meshgrid(dataset["lon"], dataset["lat"])
chl_extent = [1e-1,50]

crs_proj = cartopy.crs.Robinson()
crs_data = cartopy.crs.PlateCarree()

# loop through each 8day composite 

# set bounds
xmin, xmax = -124, -117
ymin, ymax = 32, 36
sdlonlat = [-117.24556, 32.928333]
chl_extent = [1e-1,50]


for ff in comp8d9km_files: 
    dataset = xr.open_dataset(path+ff) 
    chl = dataset["chlor_a"]
    clon, clat = np.meshgrid(dataset["lon"], dataset["lat"])
       
    crs_proj = cartopy.crs.Robinson()
    crs_data = cartopy.crs.PlateCarree()
    
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(projection=crs_proj)
    # mesh = ax.pcolormesh(lon, lat, chl, cmap="jet", transform=crs_data) #, vmin=chl_extent[0], vmax=chl_extent[1])
    mesh = ax.pcolormesh(clon,clat,chl,cmap='jet',transform=crs_data,norm=mcolors.LogNorm(vmin=chl_extent[0],vmax=chl_extent[1]) )

    cbar = plt.colorbar(mesh, ax=ax, orientation='vertical', pad=0.05)
    cbar.set_label(r'Chlorophyll Concentration [mg/$m^3$]')
    ax.set_extent((xmin,xmax,ymin,ymax))
    ax.coastlines()
    gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True,linewidth=.5, color='gray', alpha=0.2, linestyle='-')  
    gl.top_labels = False; gl.right_labels = False 
    gl.xlocator = mticker.MultipleLocator(1) 
    gl.ylocator = mticker.MultipleLocator(1) 
    d0 = datetime.strptime(dataset.attrs.get("product_name", ff)[11:19], "%Y%m%d")
    d1 = datetime.strptime(dataset.attrs.get("product_name", ff)[20:28], "%Y%m%d")
    ax.set_title(f'8 Day Composite @ 9km for {d0.strftime("%m/%d")} - {d1.strftime("%m/%d")}')
    plt.show()



In [ ]:
# nice! next we will access the ship data in order to get ship location as we go 

### Access ship data from cruise drive


In [7]:
# location of cruise folder on my local computer 
shipinfo = '/home/vboatwright/mnt/cruise/SR2503/metacq/data/'
shipinfo = '/media/vboatwright/KBZ/SR2503_cruise/metacq/data/'

seadates = [dt.datetime(2025,2,19),dt.datetime(2025,2,20),dt.datetime(2025,2,21),dt.datetime(2025,2,22),dt.datetime(2025,2,23),dt.datetime(2025,2,24),dt.datetime(2025,2,25),dt.datetime(2025,2,26),dt.datetime(2025,2,27),dt.datetime(2025,2,28),dt.datetime(2025,3,1),dt.datetime(2025,3,2)]
longdays = [sd.strftime("%Y%m%d") for sd in seadates]
days = [sd.strftime("%y%m%d") for sd in seadates]
time = [int(dd) for dd in longdays]

NN = len(days) 
slons = []
slats = []

for ii in np.arange(0,NN): 
    shipdata = np.loadtxt(shipinfo+days[ii]+'.MET',usecols=(0,24,25))
    if shipdata[1,2]>0 or shipdata[1,2]<-100:
        shipdata=np.loadtxt(shipinfo+days[ii]+'.MET',usecols=(0,24,25))
    metatime=min(shipdata[:,0], key=lambda x:abs(x-time[ii]))
    subset=shipdata[shipdata[:,0]==metatime,:]
    lat,lon=subset[0,1:3]
    slons.append(lon)
    slats.append(lat)



In [ ]:
seadates

In [ ]:
day4kmfile[0]

In [13]:

fig = plt.figure(figsize=(10,5), layout="constrained")
numdays = NN 
numday = np.arange(0,NN)
spec = fig.add_gridspec(2,numdays)

cmaps = ['RdBu_r', 'viridis']

# set bounds
xmin, xmax = -124, -117
ymin, ymax = 32, 36
sdlonlat = [-117.24556, 32.928333]

comp8d9km_file = [f for f in sorted_files if fnmatch.fnmatch(f, '*8D*9km*.nc')][-1]
dataset = xr.open_dataset(path+comp8d9km_file) 
chl = dataset["chlor_a"]
clon, clat = np.meshgrid(dataset["lon"], dataset["lat"])
chl_extent = [1e-1,50]

crs_proj = cartopy.crs.Robinson()
crs_data = cartopy.crs.PlateCarree()

# consolidate top row for 8day composite map 
ax0 = fig.add_subplot(spec[0, :],projection=crs_proj)

# mesh = ax.pcolormesh(lon, lat, chl, cmap="jet", transform=crs_data) #, vmin=chl_extent[0], vmax=chl_extent[1])
mesh = ax0.pcolormesh(clon,clat,chl,cmap='jet',transform=crs_data,norm=mcolors.LogNorm(vmin=chl_extent[0],vmax=chl_extent[1]) )

cax0 = ax0.inset_axes([1.02, 0.1, 0.05, 0.85]) # [left, bottom, width, height]
chlcbar = fig.colorbar(mesh, cax=cax0,orientation='vertical')
chlcbar.set_label(r'Chlorophyll Concentration [mg/$m^3$]')
ax0.set_extent((xmin,xmax,ymin,ymax))
ax0.coastlines()

gl = ax0.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True,linewidth=.5, color='gray', alpha=0.2, linestyle='-')  
gl.top_labels = False; gl.right_labels = False 
gl.xlocator = mticker.MultipleLocator(1) 
gl.ylocator = mticker.MultipleLocator(1) 

d0 = datetime.strptime(dataset.attrs.get("product_name", comp8d9km_file)[11:19], "%Y%m%d")
d1 = datetime.strptime(dataset.attrs.get("product_name", comp8d9km_file)[20:28], "%Y%m%d")
ax0.set_title(f'8 Day Composite @ 9km for {d0.strftime("%m/%d")} - {d1.strftime("%m/%d")}')

# add ship trajectory
numdays = np.arange(0,NN)
j = ax0.plot(slons,slats,linewidth=1,label='Ship Track',color='k',transform=crs_data)
traj = ax0.scatter(slons,slats,c=numdays,s=40,linewidth=1,transform=crs_data,cmap='hsv')

j1 = ax0.scatter(slons[0],slats[0],c='tab:red',marker='*',s=60,transform=crs_data)
ax0.annotate('Cruise\nStart',(slons[0]-0.075,slats[0]-0.25),color='tab:red',fontsize=8,fontweight='bold')


# trajectory colorbar
cax1 = ax0.inset_axes([0.4, 1.04, 0.8, 0.04]) # [left, bottom, width, height]
trajcbar = fig.colorbar(traj, cax=cax1,orientation='horizontal')
cax1.tick_params(axis='both',which='both',right=True,left=False,labelright=True)
cax1.xaxis.set_ticks_position('top'); cax1.xaxis.set_label_position('top') 
dates = [dt.datetime.strftime(d,'%m-%d') for d in seadates]
cax1.xaxis.set_ticks(numdays,labels=dates)
trajcbar.set_label('Dates along track',labelpad=5,fontsize=10)

chl_extent = [0, 3]

# make this automattically detect available files and only plot that number of subplots

for ii in np.arange(0,NN) : 
    ax1 = fig.add_subplot(spec[1,ii],projection=crs_proj)
    datenum = seadates[ii].strftime("%Y%m%d")
    day9kmfile = [f for f in sorted_files if fnmatch.fnmatch(f, f'*{datenum}*DAY*9km*.nc')]
    day4kmfile = [f for f in sorted_files if fnmatch.fnmatch(f, f'*{datenum}*DAY*4km*.nc')]
    res = '4km'
    file = day4kmfile
    dataset = xr.open_dataset(path+day4kmfile[0])
    chl = dataset["chlor_a"].values
    clon, clat = np.meshgrid(dataset["lon"], dataset["lat"])
    title = f'Daily {res} \n on {seadates[ii].strftime("%Y-%m-%d")}'
    title = f'{seadates[ii].strftime("%Y-%m-%d")}'
    
    dayplot = ax1.pcolormesh(clon, clat, chl, cmap="jet", vmin=chl_extent[0], vmax=chl_extent[1], transform=crs_data)

    ax1.set_extent((xmin, xmax, ymin, ymax))
    ax1.coastlines()
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color="gray", alpha=0.2, linestyle="-")
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.MultipleLocator(1)
    gl.ylocator = mticker.MultipleLocator(1)
    
    # reference ship  
    ax1.scatter(slons[ii], slats[ii], color="red", marker="*", s=100, transform=crs_data)
    ax1.set_title(title)

    if ii == NN : 
        # add colorbar for all plots 
        cax = ax1.inset_axes([0.1, -0.1, 0.65, 0.05]) # [left, bottom, width, height]
        chlcbar = fig.colorbar(dayplot, cax=cax,orientation='horizontal')
        chlcbar.set_label(r'Chlorophyll Concentration \n [mg/m^3]')
        

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/media/vboatwright/KBZ/SR2503_scienceparty_share/CTD/AQUA_MODIS.20250226_20250305.L3m.8D.CHL.chlor_a.9km.NRT.nc'

<Figure size 1000x500 with 0 Axes>

In [10]:
# add chl data from cruise

# next, process in a loop to plot all 

path = '/media/vboatwright/KBZ/SR2503_scienceparty_share/CTD/'
deploy = 'SurveyStations/'
cruise = 'SR2503' 

# make a variable for survey stations: 
num_stations = 13

num_variables = 17
variable_order = ['Temp','Cond','Pressure','Temp2','Cond2','Transmissometer','Fluorometer','Alimeter','PAR','Oxygen','UPV','SPAR','Salinity','Salinity2','Lon','Lat','Flag']
# longest (?) shape of p/t/c object: 212103
deepest = 222873
stations = [1,5,9,11,7,2,6,10,12,8,4,3];
cardinallocs = np.array([0,9,5,1,3,7,10,6,2,4,8,12,11]) 
cardinal = cardinallocs[1:]
survey_data = np.zeros((num_stations,num_variables,deepest))*np.nan

# loop through survey data 

castnumber = np.arange(1,14) # stations 0-13 (includes wildfire station) 
stations = [0,1,5,9,11,7,2,6,10,12,8,4,3];


for ii,cc in enumerate(castnumber):
    filename = f'{cruise}_cast_{cc:02d}_updown.cnv' # 2-digits for cast number
    if cc == 2: 
        filename = f'{cruise}_cast_{cc:02d}a_updown.cnv' # 2-digits for cast number
        
    metadata, data = parse_cnv_file(path+deploy+filename)

    # save data with index as each station #
    ss = stations[ii]

    zlen = len(data['Column_2'].values)
    
    temp1 = data['Column_0'].values
    survey_data[ss,0,0:zlen] = temp1
    
    cond1 = data['Column_1'].values
    survey_data[ss,1,0:zlen] = cond1

    p = data['Column_2'].values
    survey_data[ss,2,0:zlen] = p

    temp2 = data['Column_3'].values
    survey_data[ss,3,0:zlen] = temp2

    cond2 = data['Column_4'].values
    survey_data[ss,4,0:zlen] = cond2
    
    trans = data['Column_5'].values
    survey_data[ss,5,0:zlen] = trans
    
    fluoro = data['Column_6'].values
    survey_data[ss,6,0:zlen] = fluoro
    
    altimeter = data['Column_7'].values
    survey_data[ss,7,0:zlen] = altimeter

    PAR = data['Column_8'].values
    survey_data[ss,8,0:zlen] = PAR
    
    oxygen = data['Column_9'].values
    survey_data[ss,9,0:zlen] = oxygen
    
    upv = data['Column_10'].values
    survey_data[ii,10,0:zlen] = upv
    
    SPAR = data['Column_11'].values
    survey_data[ss,11,0:zlen] = SPAR
    
    sal1 = data['Column_12'].values
    survey_data[ss,12,0:zlen] = sal1
    
    sal2 = data['Column_13'].values
    survey_data[ss,13,0:zlen] = sal2
    
    lon = data['Column_14'].values
    survey_data[ss,14,0:zlen] = lon
    
    lat = data['Column_15'].values
    survey_data[ss,15,0:zlen] = lat

    flag = data['Column_16'].values
    survey_data[ss,16,0:zlen] = flag
    
    



NameError: name 'parse_cnv_file' is not defined

In [15]:
path = '/media/vboatwright/KBZ/SR2503_scienceparty_share/SAT//L3m_CHL_NRT_StLucia/'

In [21]:

fig = plt.figure(figsize=(20,10), layout="constrained")
numdays = NN 
numday = np.arange(0,NN)
spec = fig.add_gridspec(2,numdays)

cmaps = ['RdBu_r', 'viridis']

# set bounds
xmin, xmax = -124, -117
ymin, ymax = 32, 36
sdlonlat = [-117.24556, 32.928333]

# set new bounds
xmin, xmax = -123, -120
ymin, ymax = 34, 35.5
sdlonlat = [-117.24556, 32.928333]

comp8d9km_file = [f for f in sorted_files if fnmatch.fnmatch(f, '*8D*9km*.nc')][-1]
dataset = xr.open_dataset(path+comp8d9km_file) 
chl = dataset["chlor_a"]
clon, clat = np.meshgrid(dataset["lon"], dataset["lat"])
chl_extent = [1e-1,50]

crs_proj = cartopy.crs.Robinson()
crs_data = cartopy.crs.PlateCarree()

# consolidate top row for 8day composite map 
ax0 = fig.add_subplot(spec[0, :],projection=crs_proj)

# mesh = ax.pcolormesh(lon, lat, chl, cmap="jet", transform=crs_data) #, vmin=chl_extent[0], vmax=chl_extent[1])
mesh = ax0.pcolormesh(clon,clat,chl,cmap='jet',transform=crs_data,norm=mcolors.LogNorm(vmin=chl_extent[0],vmax=chl_extent[1]) )

cax0 = ax0.inset_axes([1.02, 0.1, 0.05, 0.85]) # [left, bottom, width, height]
chlcbar = fig.colorbar(mesh, cax=cax0,orientation='vertical')
chlcbar.set_label(r'Chlorophyll \n Concentration [mg/$m^3$]')
ax0.set_extent((xmin,xmax,ymin,ymax))
ax0.coastlines()

gl = ax0.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True,linewidth=.5, color='gray', alpha=0.2, linestyle='-')  
gl.top_labels = False; gl.right_labels = False 
gl.xlocator = mticker.MultipleLocator(1) 
gl.ylocator = mticker.MultipleLocator(1) 

d0 = datetime.strptime(dataset.attrs.get("product_name", comp8d9km_file)[11:19], "%Y%m%d")
d1 = datetime.strptime(dataset.attrs.get("product_name", comp8d9km_file)[20:28], "%Y%m%d")
ax0.set_title(f'8 Day Composite @ 9km for {d0.strftime("%m/%d")} - {d1.strftime("%m/%d")}')

# add ship trajectory
numdays = np.arange(0,NN)
j = ax0.plot(slons,slats,linewidth=1,label='Ship Track',color='k',transform=crs_data)
traj = ax0.scatter(slons,slats,c=numdays,s=40,linewidth=1,transform=crs_data,cmap='hsv')

j1 = ax0.scatter(slons[0],slats[0],c='tab:red',marker='*',s=60,transform=crs_data)
ax0.annotate('Cruise\nStart',(slons[0]-0.075,slats[0]-0.25),color='tab:red',fontsize=8,fontweight='bold')


# trajectory colorbar
cax1 = ax0.inset_axes([0.05, 1.04, 1.1, 0.04]) # [left, bottom, width, height]
trajcbar = fig.colorbar(traj, cax=cax1,orientation='horizontal')
cax1.tick_params(axis='both',which='both',right=True,left=False,labelright=True)
cax1.xaxis.set_ticks_position('top'); cax1.xaxis.set_label_position('top') 
dates = [dt.datetime.strftime(d,'%m-\n%d') for d in seadates]
cax1.xaxis.set_ticks(numdays,labels=dates)
trajcbar.set_label('Dates along track',labelpad=5,fontsize=10)

chl_extent = [0, 3]


plt.show()

save_loc = '/home/vboatwright/OneDrive/Documents/SIO/projects/santalucia/figures/cruise_report_figures/'
fig.savefig(save_loc+'oceancolor_event_visualize_track_zoom.png',dpi=300)

/home/vboatwright/python_environments/sio/lib/python3.12/site-packages/IPython/core/pylabtools.py:170: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  fig.canvas.print_figure(bytes_io, **kw)
/home/vboatwright/python_environments/sio/lib/python3.12/site-packages/IPython/core/pylabtools.py:170: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  fig.canvas.print_figure(bytes_io, **kw)


/tmp/ipykernel_1876174/3947173125.py:72: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  fig.savefig(save_loc+'oceancolor_event_visualize_track_zoom.png',dpi=300)
/tmp/ipykernel_1876174/3947173125.py:72: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  fig.savefig(save_loc+'oceancolor_event_visualize_track_zoom.png',dpi=300)


In [ ]:
import matplotlib.gridspec as gridspec

# use most recent saved chl map and ship track 
# inset the daily chl from sat at bottom 

dataset = xr.open_dataset(path+ff) 
chl = dataset["chlor_a"]
clon, clat = np.meshgrid(dataset["lon"], dataset["lat"])

crs_proj = cartopy.crs.Robinson()
crs_data = cartopy.crs.PlateCarree()

fig = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(1,2,width_ratios=[15,1],wspace=0.1)
ax = fig.add_subplot(gs[0],projection=crs_proj)

# mesh = ax.pcolormesh(lon, lat, chl, cmap="jet", transform=crs_data) #, vmin=chl_extent[0], vmax=chl_extent[1])
mesh = ax.pcolormesh(clon,clat,chl,cmap='jet',transform=crs_data,norm=mcolors.LogNorm(vmin=chl_extent[0],vmax=chl_extent[1]) )

cbar_ax = fig.add_subplot(gs[1])
cbar = plt.colorbar(mesh, ax=cbar_ax, orientation='vertical') #, fraction=0.046, pad=0.02)
cbar.set_label(r'Chlorophyll Concentration [mg/$m^3$]')
ax.set_extent((xmin,xmax,ymin,ymax))
ax.coastlines()

gl = ax.gridlines(crs=cartopy.crs.PlateCarree(), draw_labels=True,linewidth=.5, color='gray', alpha=0.2, linestyle='-')  
gl.top_labels = False; gl.right_labels = False 
gl.xlocator = mticker.MultipleLocator(1) 
gl.ylocator = mticker.MultipleLocator(1) 

d0 = datetime.strptime(dataset.attrs.get("product_name", ff)[11:19], "%Y%m%d")
d1 = datetime.strptime(dataset.attrs.get("product_name", ff)[20:28], "%Y%m%d")
ax.set_title(f'8 Day Composite @ 9km for {d0.strftime("%m/%d")} - {d1.strftime("%m/%d")}')

# add ship trajectory
numdays = np.arange(0,NN)
j = ax.plot(slons,slats,linewidth=1,label='Ship Track',color='k',transform=crs_data)
k = ax.scatter(slons,slats,c=numdays,s=4,linewidth=1,transform=crs_data)

j1 = ax.scatter(slons[0],slats[0],c='tab:red',marker='*',s=60,transform=crs_data)
ax.annotate('Cruise\nStart',(slons[0]-0.075,slats[0]-0.25),color='tab:red',fontsize=8,fontweight='bold')

# plt.legend()

# add colorbars for plot to show time of trajectory

'''
cax = fig.add_axes([0.825,0.075,0.015,0.4]) # [left, bottom, width, height]
timerange = np.linspace(ts[0],ts[-1],8)
dates = [datetime.datetime.utcfromtimestamp(d) for d in timerange]
dates = [datetime.datetime.strftime(d,'%m-%d') for d in dates]
cbar = plt.colorbar(j,cax=cax,orientation='vertical',ticks=numdays) 
cax.tick_params(axis='both',which='both',left=False,right=True,labelright=False)
'''

# cax = fig.add_axes([0.845,0.075,0.015,0.4]) # [left, bottom, width, height]
cax2 = fig.add_axes([0.6, 0.15, 0.02, 0.7])  
dates = [dt.datetime.strftime(d,'%m-%d') for d in seadates]
cbar = plt.colorbar(k,cax=cax2,orientation='horizontal')
cax2.yaxis.set_ticks(numdays,labels=dates)
cax2.tick_params(axis='both',which='both',right=True,left=False,labelright=True)
cbar.set_label('Dates along track',labelpad=10,fontsize=10)
#cax.xaxis.set_ticks_position('top') 
#cax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))


plt.show()



In [ ]:
fig = plt.figure(figsize=(10,5), layout="constrained")
numdays = NN 
numday = np.arange(0,NN)
spec = fig.add_gridspec(2,numdays)

cmaps = ['RdBu_r', 'viridis']

# consolidate top row 
ax0 = fig.add_subplot(spec[0, :])
pcm = ax0.pcolormesh(np.random.random((20, 20)) * (col + 1),cmap=cmaps[1])
traj = ax0.scatter([0,1,2,3],[10,10,10,10],c=numday,s=4,linewidth=1)
# chl colorbar 
cax0 = ax0.inset_axes([1.02, 0.1, 0.05, 0.8]) # [left, bottom, width, height]
chlcbar = fig.colorbar(pcm, cax=cax0,orientation='vertical')
# trajectory colorbar
cax1 = ax0.inset_axes([0.8, 1.02, 0.21, 0.05]) # [left, bottom, width, height]
trajcbar = fig.colorbar(traj, cax=cax1,orientation='horizontal')
cax1.tick_params(axis='both',which='both',right=True,left=False,labelright=True)
cax1.xaxis.set_ticks_position('top'); cax1.xaxis.set_label_position('top') 
trajcbar.set_label('Dates along track',labelpad=5,fontsize=10)


for ii in np.arange(0,numdays) : 
    ax = fig.add_subplot(spec[1,ii])
    
fig.suptitle('Manually added subplots, spanning a column')

In [ ]:
[f for f in sorted_files if fnmatch.fnmatch(f, '*20250220*DAY*9km*.nc')]

In [ ]:
day9kmfile = [f for f in sorted_files if fnmatch.fnmatch(f, f'*{datenum}*DAY*9km*.nc')]
day9kmfile

In [ ]:

fig, axs = plt.subplots(2, 2)
cmaps = ['RdBu_r', 'viridis']
for col in range(2):
    for row in range(2):
        ax = axs[row, col]
        pcm = ax.pcolormesh(np.random.random((20, 20)) * (col + 1),
                            cmap=cmaps[col])
        fig.colorbar(pcm, ax=ax)



In [ ]:
fig, axs = plt.subplots(3, 3, layout='constrained')
for ax in axs.flat:
    pcm = ax.pcolormesh(np.random.random((20, 20)))

fig.colorbar(pcm, ax=axs[0, :2], shrink=0.6, location='bottom')
fig.colorbar(pcm, ax=[axs[0, 2]], location='bottom')
fig.colorbar(pcm, ax=axs[1:, :], location='right', shrink=0.6)
fig.colorbar(pcm, ax=[axs[2, 1]], location='left')

In [ ]:
numdays = NN 

fig, axs = plt.subplots(2, numdays, layout='constrained')
gs = axs[0, 2].get_gridspec()
# remove top axes to aggregate into 1 subplot for map 


cmaps = ['RdBu_r', 'viridis']

for col in range(2):
    for row in range(2):
        ax = axs[row, col]
        pcm = ax.pcolormesh(np.random.random((20, 20)) * (col + 1),
                            cmap=cmaps[col])
        if col == 0:
            ax.set_aspect(2)
        else:
            ax.set_aspect(1/2)
        if row == 1:
            cax = ax.inset_axes([1.04, 0.2, 0.05, 0.6])
            fig.colorbar(pcm, cax=cax)

In [ ]:
fig = plt.figure(layout="constrained")
subfigs = fig.subfigures(1, 2, wspace=0.07, width_ratios=[1.5, 1.])
axs0 = subfigs[0].subplots(2, 2)
subfigs[0].set_facecolor('lightblue')
subfigs[0].suptitle('subfigs[0]\nLeft side')
subfigs[0].supxlabel('xlabel for subfigs[0]')

axs1 = subfigs[1].subplots(3, 1)
subfigs[1].suptitle('subfigs[1]')
subfigs[1].supylabel('ylabel for subfigs[1]')
